<a href="https://colab.research.google.com/github/Alepescinaa/ScientificTools/blob/main/Project1/Cp2/PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercises: physics-informed neural network

Exercise on the implementation of physics-informed neural network.

Date: 2024

Course: 056936 - SCIENTIFIC COMPUTING TOOLS FOR ADVANCED MATHEMATICAL MODELLING (PAGANI STEFANO) [2023-24].

Example adapted from this [notebook](https://colab.research.google.com/drive/1qBrbgevkSBqqYc8bOPiaoJG1MBrBrluN?usp=share_link).


Let us consider the problem

\begin{aligned}
  & v_f *\sqrt(\nabla u\cdot D\nabla u) =1  \,, \quad (x,y) \in [-1.5,1.5] \times [-1.5,1.5]\,\\
\end{aligned}

where $\nu$ is unknown. We consider the PINN framework for solving the state/parameter estimation.

In [ ]:
# import required libraries

import tensorflow as tf
import numpy as np
import scipy.io
from tensorflow import keras
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
!pip -q install pyDOE
from pyDOE import lhs  # for latin hypercube sampling

  Preparing metadata (setup.py) ... done


In [ ]:
# set seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
!git clone https://github.com/Alepescinaa/ScientificTools
%cd ScientificTools/Project1/Cp2

Cloning into 'ScientificTools'...
remote: Enumerating objects: 374, done.
remote: Counting objects: 100% (218/218), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 374 (delta 64), reused 1 (delta 1), pack-reused 156
Receiving objects: 100% (374/374), 99.95 MiB | 11.25 MiB/s, done.
Resolving deltas: 100% (102/102), done.
/content/ScientificTools/Project1/Cp2


In [ ]:
# loading of the dataset

CP2data = np.load("CP2data.npz")
CP2data = CP2data['arr_0']

In [ ]:
# loading of the estimate

CP2estimate = np.load("CP2estimate.npz")
CP2estimate = CP2estimate['arr_0']

In [ ]:
CP2estimate[4]

array([0.1226488 , 3.6316356 , 0.84714825])

# Solution

In [ ]:
X, Y = np.meshgrid(np.linspace(-1.5,1.5,151), np.linspace(-1.5,1.5,151))
X_flat = tf.convert_to_tensor(np.hstack((X.flatten()[:,None],Y.flatten()[:,None])),dtype=tf.float64)
x0 = 1.5
#activation_time = anysotropic_FMM( x0 , y0 , X, Y, sigma_11,sigma_12, sigma_21, sigma_22 )

In [ ]:
def penalty(param, lower_bound, upper_bound):
    return tf.reduce_sum(tf.square(tf.maximum(param - upper_bound, 0)) +
                         tf.square(tf.maximum(lower_bound - param, 0)))
# PINN loss function
def loss(xcl,ycl,xmeas,ymeas,umeas,param):
    umeas_pred = PINN(tf.concat([xmeas,ymeas],1))
    r_pred   = r_PINN(xcl,ycl,param)

    # loss components
    mse_meas  = tf.reduce_mean(tf.pow(umeas-umeas_pred,2))
    mse_r  = tf.reduce_mean(tf.pow(r_pred,2))

    # bc
    mse_bc= tf.pow( PINN( tf.transpose( tf.stack( [tf.constant([1.5],dtype=tf.float64), param[2] ] ) ) ) ,2)

    #penalty over param boundaries
    mse_penalty=penalty(param[0],-np.pi/10,np.pi/10)+penalty(param[1],1,9)+penalty(param[2],-1.5,1.5)

    return mse_r + mse_meas + mse_bc + mse_penalty
    #tf.print('mse_time',mse_meas)
    #tf.print('mse_param',mse_r)
    #tf.print('mse_bc',mse_bc)

# neural network weight gradients
@tf.function
def grad(model,xcl,ycl,xmeas,ymeas,umeas,param):
    with tf.GradientTape(persistent=True) as tape:
        loss_value = loss(xcl,ycl,xmeas,ymeas,umeas,param)
        grads = tape.gradient(loss_value,model.trainable_variables)
        grad_param = tape.gradient(loss_value,param)
    return loss_value, grads, grad_param

In [ ]:
from sklearn.model_selection import train_test_split
# collocation points
Ncl = 10000
Xcl = lhs(2,Ncl)
xcl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,0],dtype=tf.float64),axis=-1)
ycl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,1],dtype=tf.float64),axis=-1)
X_coll = tf.concat([xcl,ycl],1)

# measurement points
ind_disp=4
xmeas = CP2data[ind_disp][0]
ymeas = CP2data[ind_disp][1]
tmeas = CP2data[ind_disp][2]
xmeas_train, xmeas_val, ymeas_train, ymeas_val, tmeas_train, tmeas_val = train_test_split(xmeas, ymeas, tmeas, test_size=0.1)
xmeas_train = tf.constant(xmeas_train.reshape(18, 1), dtype=tf.float64)
ymeas_train = tf.constant(ymeas_train.reshape(18, 1), dtype=tf.float64)
tmeas_train = tf.constant(tmeas_train.reshape(18, 1), dtype=tf.float64)
xmeas_val = tf.constant(xmeas_val.reshape(2, 1), dtype=tf.float64)
ymeas_val = tf.constant(ymeas_val.reshape(2, 1), dtype=tf.float64)
tmeas_val = tf.constant(tmeas_val.reshape(2, 1), dtype=tf.float64)




In [ ]:
# training loop

from tensorflow.keras import regularizers

regularization_strength =1e-3

PINN = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='tanh', input_shape=(2,),
                          kernel_initializer="glorot_normal",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),
    tf.keras.layers.Dense(32, activation='tanh',
                          kernel_initializer="glorot_normal",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),
    tf.keras.layers.Dense(1, activation=None,
                          kernel_initializer="glorot_normal",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64)])

In [ ]:
# residual computation based on AD
@tf.function
def r_PINN(x,y,param):
    u = PINN(tf.concat([x,y], 1))
    u_x = tf.gradients(u,x)[0]
    u_y = tf.gradients(u,y)[0]
    u_grad = tf.transpose(tf.concat([u_x, u_y], axis=1))

    theta0 = pi/2 - param[0]
    a = tf.stack([tf.cos(theta0), tf.sin(theta0)])
    b = tf.stack([tf.cos(theta0-pi/2), tf.sin(theta0-pi/2)])

    D = ((1/param[1])* tf.linalg.matmul(a,tf.transpose(a)) + tf.linalg.matmul(b,tf.transpose(b)))

    #return  tf.linalg.matmul(tf.transpose(u_grad), tf.linalg.matmul(D,u_grad)) - 1/100**2
    return tf.sqrt((u_x * D[0,0] * u_x + u_x * D[0,1] * u_y + u_y * D[1,0] * u_x + u_y * D[1,1] * u_y))  - 1/100

# Adam optimizer
initial_learning_rate=0.005

tf_optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate,beta_1=0.99)

# parameter variable try ( 1.45816415][ 3.84494007][-0.95064015]])
#param 0 -> fiber anlge (-pi/10,pi/10)
#param 1-> aniso (0,1)
#param 2-> source y0 (-1.5,1,5)
pi = tf.constant(np.pi,dtype=tf.float64)
param = tf.Variable([[0.01], [3], [0.1]], trainable=True,dtype=tf.float64)

patience = 1000
patience_lr= 200
min_delta = 1e-9
best_val_loss = float('inf')
wait = 0

for iter in range(20000):

  # compute gradients using AD
  loss_value,grads,grad_param = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train, param)

  # update neural network weights
  tf_optimizer.apply_gradients(zip(grads+[grad_param],PINN.trainable_variables+[param]))

  val_loss, _, _ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val, param)



  # Early stopping
  if val_loss < best_val_loss - min_delta:
      best_val_loss = val_loss
      wait = 0
  else:
      wait += 1
      if wait >= patience_lr:
         tf_optimizer.learning_rate = tf_optimizer.learning_rate/2

      if wait >= patience:
          print('Early stopping at epoch', iter + 1)
          break

  # display intermediate results
  if ((iter+1) % 100 == 0):
    print('iter =  '+str(iter+1))
    #loss_value_np=loss_value.numpy()
    #print('loss = {:.4f}'.format(loss_value_np ))
    tf.print('loss =' , loss_value)
    tf.print('loss_val =' , val_loss)

    print(param.numpy())
    PINN_flat = PINN(X_flat)
    """
    mse = 0.0
    time_pred = tf.reshape(PINN_flat, (151, 151))
    time_pred = time_pred.numpy()
    xmeas=xmeas.numpy().squeeze()
    ymeas=ymeas.numpy().squeeze()
    tmeas=tmeas.numpy().squeeze()

    for k in range(20):
            i, j = np.where((X == xmeas[k]) & (Y == ymeas[k]))
            mse+= (time_pred[i, j] - tmeas[k]) ** 2

    print('mse error: %.4e' % (np.sqrt(mse/20)))

    xmeas = tf.constant(xmeas.reshape(20, 1), dtype=tf.float64)
    ymeas = tf.constant(ymeas.reshape(20, 1), dtype=tf.float64)
    tmeas = tf.constant(tmeas.reshape(20, 1), dtype=tf.float64)
    """

iter =  100
loss = [[0.012842736742309874]]
loss_val = [[0.014768665156964421]]
[[ 0.2165607 ]
 [ 3.25567154]
 [-0.03262506]]
iter =  200
loss = [[0.0016689169917409197]]
loss_val = [[0.0018782382685200191]]
[[ 0.32069147]
 [ 3.39161843]
 [-0.08389283]]
iter =  300
loss = [[0.0010691755464865243]]
loss_val = [[0.0013573127197935755]]
[[ 0.28022573]
 [ 3.46086245]
 [-0.11955686]]
iter =  400
loss = [[0.00017359604143097105]]
loss_val = [[0.00019134668355448006]]
[[ 0.27111998]
 [ 3.50012027]
 [-0.13521664]]
iter =  500
loss = [[0.00027452690259607859]]
loss_val = [[0.00023403193333612461]]
[[ 0.27051288]
 [ 3.51865938]
 [-0.13913044]]
iter =  600
loss = [[0.00011426445065841282]]
loss_val = [[8.44118324352358e-05]]
[[ 0.27452319]
 [ 3.52945907]
 [-0.13864798]]
iter =  700
loss = [[0.00011424580056568066]]
loss_val = [[8.4440971164441e-05]]
[[ 0.27452388]
 [ 3.52946033]
 [-0.13864745]]
iter =  800
loss = [[0.00011424580056568066]]
loss_val = [[8.4440971164441e-05]]
[[ 0.27452388]
 [ 3.52

In [ ]:
#Display results

N_h = 150
X_plot, Y_plot = np.meshgrid(np.linspace(-1.5,1.5,1501), np.linspace(-1.5,1.5,1501))

fig = plt.figure(figsize=(16,9),dpi=150)
#fig = plt.figure()
#fig.subplots_adjust(wspace=0.3)
plt.style.use('default')
ax = fig.add_subplot(1,3,1)
ax.set_aspect(1)
im = plt.pcolor(X_plot, Y_plot, u_true)
plt.scatter(xmeas,ymeas,marker='x',s=3)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im,cax=cax)
#ax.set_yticklabels(['-1.0','-0.6','-0.2','0.2','0.6','1.0'])
ax.set_title('Exact Solution',fontsize=16)

ax = fig.add_subplot(1,3,2)
ax.set_aspect(1)
im = plt.pcolor(X_plot, Y_plot, np.reshape(PINN_flat,(N_h,N_h)))
plt.scatter(xmeas,ymeas,marker='x',s=3)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im,cax=cax)
ax.set_title('PINN Prediction'.format(err),fontsize=16)

ax = fig.add_subplot(1,3,3)
ax.set_aspect(1)
im = plt.pcolor(X_plot, Y_plot, np.abs( np.reshape(PINN_flat,(N_h,N_h)) -u_true ) )
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im,cax=cax)
ax.set_title('L2 error = {:.4f}'.format(err),fontsize=16)